## Packages

In [ ]:
import pandas as pd
import numpy as np
from math import ceil
## import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import sqlalchemy 
from sqlalchemy import create_engine, text

import sys
import os

## Add the path of the functions folder
current_dir = os.getcwd()  ## Gets the current working directory
sub_dir = os.path.abspath(os.path.join(current_dir, '..'
                                       , 'Functions'))
sys.path.append(sub_dir)

# Now you can import functions
from db_secrets import SQL_107

In [ ]:
# TensorFlow sequential model
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Turn warnings off to keep notebook tidy
import warnings
warnings.filterwarnings("ignore")

## Connection

In [ ]:
## text for query
with open("../Exploratory_Analysis/111_sql.sql", "r") as file:
    query_text = file.read()

query_text = query_text.replace('REPLACE START DATE','2022-01-01')

In [ ]:
## Create an engine + connection
engine = create_engine(SQL_107())
conn = engine.connect()

## Return data
df_raw = pd.read_sql(query_text,conn)

## Wrangle

In [ ]:
## Makes working copy
df = df_raw.copy()

#df = df.sample(n=100000, random_state=42)

In [ ]:
## List columns
df.columns

In [ ]:
df = df[['Call Connect Time'
         ,'Bank Holiday'
         , 'In_Out_Hours'
         , 'Sub ICB Name'
         ,'Outcome Type']].copy()

#### binary outcome

In [ ]:
df['Calls'] = 1

In [ ]:
df['Outcome'] = df['Outcome Type'].transform(lambda x: 0 if x == 'No UEC Contact' else 1)
df = df.drop(['Outcome Type'],axis=1) 

In [ ]:
timestamp_s = df['Call Connect Time'].map(pd.Timestamp.timestamp)


In [ ]:
df['Call Connect Time'].dt.day_of_year

In [ ]:
## Date time conversion to numeric
df['year']    = df['Call Connect Time'].dt.year

df['month sin'] = np.sin(df['Call Connect Time'].dt.month * (2*np.pi/12))
df['month cos'] = np.cos(df['Call Connect Time'].dt.month * (2*np.pi/12))

df['YearDay sin'] = np.sin(df['Call Connect Time'].dt.day_of_year * (2*np.pi/365))
df['YearDay cos'] = np.cos(df['Call Connect Time'].dt.day_of_year * (2*np.pi/365))

df['weekday sin'] = np.sin(df['Call Connect Time'].dt.weekday+1 * (2*np.pi/7))  # Monday=0, Sunday=6
df['weekday cos'] = np.cos(df['Call Connect Time'].dt.weekday+1 * (2*np.pi/7))  # Monday=0, Sunday=6

df['Hour sin'] = np.sin(df['Call Connect Time'].dt.hour * (2*np.pi/24))
df['Hour cos'] = np.cos(df['Call Connect Time'].dt.hour * (2*np.pi/24))

df = df.drop('Call Connect Time',axis=1) 

#### Aggregates count 

In [ ]:
df = df.groupby([ 'year'
         , 'month sin'
         , 'month cos'
         , 'YearDay sin'
         , 'YearDay cos'
         , 'weekday sin'
         , 'weekday cos'
         , 'Hour sin'
         , 'Hour cos'
         , 'Bank Holiday'
         , 'In_Out_Hours'
         , 'Sub ICB Name']).agg('sum').reset_index()

In [ ]:
## One hot encodinng for boolean variables
bool_mapping = {
    'Yes': 1,
    'No': 0,
    'In Hours': 1,
    'Out of Hours': 0
}

df.loc[:,'Is Bank Holiday'] = df['Bank Holiday'].map(bool_mapping)             
df.loc[:,'In Hours'] = df['In_Out_Hours'].map(bool_mapping)
df = df.drop(['Bank Holiday','In_Out_Hours'],axis=1) 

In [ ]:
## Dummy variables from ICB
df = pd.concat([df, pd.get_dummies(df['Sub ICB Name']
                                   ,dtype=int
                                   , prefix='SubICB')]
                ,axis=1)
df = df.drop('Sub ICB Name', axis=1)

In [ ]:
df.head()

## Build a baseline mode

Baseline split

In [ ]:
base_X = df.drop('Outcome',axis=1)# X = all  except the 'Outcome' column
base_y = df['Outcome']# y = 'Outcome' column 

base_X_train, base_X_test, base_y_train, base_y_test = train_test_split(base_X
                                                    , base_y 
                                                    , test_size = 0.25
                                                    , random_state=42)

In [ ]:
## Joins outcome onto predictors
base_df = pd.concat([base_X_train,base_y_train],axis=1)

In [ ]:
## Columns we care about for baseline model
group_cols = [
    'month sin'
    ,'month cos'    
    ,'weekday sin'
    ,'weekday cos'
    ,'Hour sin'
    ,'Hour cos'
    ,'SubICB_County Durham'
    ,'SubICB_Newcastle Gateshead'
    ,'SubICB_North Tyneside'
    ,'SubICB_Northumberland'
    ,'SubICB_South Tyneside'
    ,'SubICB_Sunderland'
    ,'SubICB_Tees Valley'
]

## Mean value across baseline
base_trained = (base_df[group_cols + ['Outcome']]
                .groupby(group_cols)
                .agg( Pred_Outcome=pd.NamedAgg(column="Outcome"
                                               , aggfunc="mean"))
                )





In [ ]:
base_y_pred_train = pd.merge(base_X_train,base_trained,how='left',on=group_cols)['Pred_Outcome']
base_y_pred_test = pd.merge(base_X_test,base_trained,how='left',on=group_cols)['Pred_Outcome']

In [ ]:
def calculate_baseline_accuracy(y_pred_train,y_pred_test,y_train, y_test):
    """Calculate and print accuracy of training and test data fits"""    

    # Calculate Mean Absolute Error (MAE) for training and test sets
    mae_train = np.mean(np.abs(y_pred_train - y_train))
    mae_test = np.mean(np.abs(y_pred_test - y_test))
    
    # Calculate Mean Squared Error (MSE) for training and test sets
    mse_train = np.mean((y_pred_train - y_train) ** 2)
    mse_test = np.mean((y_pred_test - y_test) ** 2)

    # Print the results
    print(f'Training MAE: {mae_train:.3f}')
    print(f'Test MAE: {mae_test:.3f}')
    print(f'Training MSE: {mse_train:.3f}')
    print(f'Test MSE: {mse_test:.3f}')

## Split

In [ ]:
X = df.drop('Outcome',axis=1).to_numpy() # X = all  except the 'Outcome' column
y = df['Outcome'].to_numpy() # y = 'Outcome' column 


X_train, X_test, y_train, y_test = train_test_split(X
                                                    , y 
                                                    , test_size = 0.25
                                                    , random_state=42)

## Scale

In [ ]:
def scale_data(Xy_train, Xy_test,X_or_y = ['X','y']):
    """Scale data 0-1 based on min and max in training set"""
    
    # Initialise a new scaling object for normalising input data
    sc = MinMaxScaler()
    
    if X_or_y == 'X':
        # Apply the scaler to the training and test sets
        train_sc = sc.fit_transform(Xy_train)
        test_sc = sc.transform(Xy_test)

    elif X_or_y == 'y':
        # Apply the scaler to the training and test sets
        train_sc = sc.fit_transform(Xy_train.reshape(-1, 1))
        test_sc = sc.transform(Xy_test.reshape(-1, 1))        
        
    return train_sc, test_sc

In [ ]:
# Scale X data
X_train, X_test = scale_data(X_train, X_test, X_or_y='X')


# Scale y data
#y_train, y_test = scale_data(y_train, y_test, X_or_y='y')

## Build a model

In [ ]:
def make_net(number_features, 
             hidden_layers=3, 
             hidden_layer_neurones=128, 
             dropout=0.0, 
             learning_rate=0.003):
    
    """Make TensorFlow neural net"""
    
    # Clear Tensorflow 
    K.clear_session()
    
    # Set up neural net
    net = Sequential()
    
    # Add hidden hidden_layers using a loop
    for i in range(hidden_layers):
        # Add fully connected layer with ReLu activation
        net.add(Dense(
            hidden_layer_neurones, 
            input_dim=number_features,
            activation='relu'))
        # Add droput layer
        net.add(Dropout(dropout))
    
    # Add final sigmoid activation output
    net.add(Dense(1, activation='linear'))    
    #    net.add(Dense(1, activation='sigmoid'))    

    # Compiling model
    opt = Adam(learning_rate=learning_rate)
    
    net.compile(loss='mse', 
                optimizer=opt, 
                metrics=['mae'])
    
    return net

In [ ]:
def calculate_accuracy(model, X_train_sc, X_test_sc, y_train, y_test):
    """Calculate and print accuracy of training and test data fits"""    
    
   # Predict on training and test data
    y_pred_train = model.predict(X_train_sc).flatten()
    y_pred_test = model.predict(X_test_sc).flatten()

    # Calculate Mean Absolute Error (MAE) for training and test sets
    mae_train = np.mean(np.abs(y_pred_train - y_train))
    mae_test = np.mean(np.abs(y_pred_test - y_test))
    
    # Calculate Mean Squared Error (MSE) for training and test sets
    mse_train = np.mean((y_pred_train - y_train) ** 2)
    mse_test = np.mean((y_pred_test - y_test) ** 2)

    # Print the results
    print(f'Training MAE: {mae_train:.3f}')
    print(f'Test MAE: {mae_test:.3f}')
    print(f'Training MSE: {mse_train:.3f}')
    print(f'Test MSE: {mse_test:.3f}')

In [ ]:
def plot_training(history_dict,measure='mae'):
    acc_values = history_dict[measure]
    val_acc_values = history_dict[f'val_{measure}']
    epochs = range(1, len(acc_values) + 1)

    fig, ax = plt.subplots()

    ax.set_xlabel('Time')
    ax.set_ylabel(measure)

    ax.plot(epochs, acc_values, color='blue', label=f'Training {measure}')
    ax.plot(epochs, val_acc_values, color='red', label=f'Test {measure}')
    ax.set_title(f'Training and validation {measure}')
    
    ax.legend()

    fig.show()

In [ ]:
model = make_net(number_features=X_train.shape[1], 
             hidden_layers=3, 
             hidden_layer_neurones=128, 
             dropout=0.10, 
             learning_rate=0.003)

In [ ]:
# Define save checkpoint callback (only save if new best validation results)
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.keras', save_best_only=True)

# Define early stopping callback
# Stop when no validation improvement for 25 epochs
# Restore weights to best validation accuracy
early_stopping_cb_loss = keras.callbacks.EarlyStopping(
    patience=20, restore_best_weights=True, monitor='val_loss')

#early_stopping_cb_acc = keras.callbacks.EarlyStopping(
#    patience=5, restore_best_weights=True, monitor='val_accuracy')


### Train model (and store training info in history)
history = model.fit(X_train,
                    y_train,
                    epochs=200,
                    batch_size=64,
                    validation_data=(X_test, y_test),
                    verbose=1,
                    callbacks=[checkpoint_cb
                               , early_stopping_cb_loss
                               #, early_stopping_cb_acc
                               ])

## Accuracy

In [ ]:
## Baseline accuracy is the mean of site, month, weekday, hour
calculate_baseline_accuracy(base_y_pred_train
                            ,base_y_pred_test
                            ,base_y_train
                            ,base_y_test)


In [ ]:
calculate_accuracy(model, X_train, X_test, y_train, y_test)

In [ ]:
plot_training(history.history,measure='mae')

In [ ]:
def plot_prediction_error(y_true, y_pred, title='Prediction Error Plot'):
    """Create a scatter plot comparing predicted and actual values, with an error line."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Scatter plot of actual vs predicted values
    ax.scatter(y_true, y_pred, edgecolors=(0, 0, 0), alpha=0.7)
    ax.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=2, color='red')
    
    ax.set_xlabel('Actual Values')
    ax.set_ylabel('Predicted Values')
    ax.set_title(title)
    ax.grid(True)
    plt.show()

# Generate predictions
y_pred_train = model.predict(X_train).flatten()
y_pred_test = model.predict(X_test).flatten()

# Plot errors for both training and test data
plot_prediction_error(y_train, y_pred_train, title='Training Data - Prediction Error Plot')
plot_prediction_error(y_test, y_pred_test, title='Test Data - Prediction Error Plot')

In [ ]:
def plot_prediction_density_subplots(y_train, y_pred_train, y_test, y_pred_test):
    """Create two subplots with shared axes and a common logarithmic color scale."""
    fig, ax = plt.subplots(1, 2, figsize=(14, 6), sharex=True, sharey=True)
    
    # Create hexbin plot for training data
    hb1 = ax[0].hexbin(y_train, y_pred_train, gridsize=50, cmap='Blues', mincnt=1, 
                       norm=mcolors.LogNorm())
    ax[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 
               'k--', lw=2, color='red')
    ax[0].set_title('Training Data - Prediction Density Plot')
    ax[0].set_xlabel('Actual Values')
    ax[0].set_ylabel('Predicted Values')
    ax[0].grid(True)

    # Create hexbin plot for test data
    hb2 = ax[1].hexbin(y_test, y_pred_test, gridsize=50, cmap='Blues', mincnt=1, 
                       norm=mcolors.LogNorm())
    ax[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
               'k--', lw=2, color='red')
    ax[1].set_title('Test Data - Prediction Density Plot')
    ax[1].set_xlabel('Actual Values')
    ax[1].grid(True)

    # Add a single colorbar for both subplots
    cb = fig.colorbar(hb1, ax=ax, orientation='vertical', fraction=0.03, pad=0.04)
    cb.set_label('Log Density')

    plt.tight_layout()
    plt.show()

# Generate predictions
y_pred_train = model.predict(X_train).flatten()
y_pred_test = model.predict(X_test).flatten()

# Plot the density plots as subplots
plot_prediction_density_subplots(y_train, y_pred_train, y_test, y_pred_test)
